In [755]:
import numpy as np
import pandas as pd
from math import radians, pi
from numpy import cos, sin, arcsin, sqrt

In [756]:
df= pd.read_csv('C:\\repos\\DataSample.csv',sep=',',header=0)
df.set_index('_ID', inplace=True)

In [757]:
dfPoi = pd.read_csv('C:\\repos\\POIList.csv',sep=',',header=0)

In [758]:
df = df.drop_duplicates(subset=['Latitude','Longitude'])

In [759]:
rlat = df.iloc[0,4]
rlon = df.iloc[0,5]
plat = dfPoi.iloc[0,1]
plon = dfPoi.iloc[0,2]

In [760]:
def haversine(rlat, rlon, plat, plon):
    rlat, rlon, plat, plon = map(radians, [rlat, rlon, plat, plon])
    dlon = plon - rlon 
    dlat = plat - rlat 
    a = sin(dlat/2)**2 + cos(plat) * cos(rlat) * sin(dlon/2)**2
    c = 2 * arcsin(sqrt(a)) 
    km = 6367 * c
    return km

In [761]:
otcnt = 0
thrcnt = 0
frcnt = 0
otsum = 0
thrsum = 0
frsum = 0
pot = []
pth = []
pfr = []
lst = []
dlst = []
alst = []
slst = []
otlst=[]
thlst = []
frlst = []
x=0
r=dfPoi.iloc[0,0]

In [762]:
for rval, rrow in df.iterrows():
    min = haversine(rlat, rlon, df.iloc[x,4], df.iloc[x,5])
    x += 1
    for pval, prow in dfPoi.iterrows():
        dist = haversine(rrow.iloc[4],rrow.iloc[5], prow.iloc[1], prow.iloc[2])
        if dist <= min:
            min = dist
            r = prow.iloc[0]
    if r == 'POI3':
        thrcnt = thrcnt +1
        thrsum = thrsum + dist
        pth.append(dist)
        thlst.append('POI3')
        thrCircledf = pd.DataFrame({'POID':thlst,'Radius(Km)':pth})
        lst.append('POI3')
        dlst.append(dist)
    elif r == 'POI4':
        frcnt = frcnt + 1
        frsum = frsum + dist
        pfr.append(dist)
        frlst.append('POI4')
        lst.append('POI4')
        dlst.append(dist)
    elif r == 'POI1' or r == 'POI2': 
        otcnt = otcnt + 1 
        otsum = otsum + dist
        pot.append(dist)
        otlst.append('POI1 or POI2')
        lst.append('POI1 or POI2')
        dlst.append(dist)
    else: print('ERROR DISTANCE', dist, 'POI', r)

In [763]:
otCircledf = pd.DataFrame({'POID':otlst, 'Radius(Km)':pot})
frCircledf = pd.DataFrame({'POID':frlst, 'Radius(Km)':pfr})
thrCircledf = pd.DataFrame({'POID':thlst,'Radius(Km)':pth})

In [764]:
RadiusPOI1 = otCircledf['Radius(Km)'].max()
RadiusPOI2 = RadiusPOI1
RadiusPOI4 = frCircledf['Radius(Km)'].max()
RadiusPOI3 = thrCircledf['Radius(Km)'].max()
POI1Density = (2.0*RadiusPOI1*pi/otCircledf.count()).astype(float).iloc[1]
POI2Density = POI1Density
POI3Density = (2.0*RadiusPOI3*pi/thrCircledf.count()).astype(float).iloc[1]
POI4Density = (2.0*RadiusPOI3*pi/frCircledf.count()).astype(float).iloc[1]

In [765]:
alst = [np.average(pot), np.average(pot), np.average(pth), np.average(pfr)]
slst = [np.std(pot), np.std(pot), np.std(pth), np.std(pfr)]

In [766]:
dfPoi['AverageDistance (Km)'] = alst
dfPoi['Standard Dev'] = slst
dfPoi['Density'] = [POI1Density, POI2Density, POI3Density, POI4Density]

In [767]:
df['Closest POI'] = lst
df['Distance(Km)'] = dlst

In [768]:
df.to_csv("C:\\repos\\dataout.csv")

In [777]:
print('For each POI, we calculated the Average Distance in Km and standard deviation of the distance between the POI to each of its assigned requests.',)
print('The density is calculated (requests\/area) for each POI.')
dfPoi

For each POI, we calculated the Average Distance in Km and standard deviation of the distance between the POI to each of its assigned requests.
The density is calculated (requests\/area) for each POI.


,POIID,Latitude,Longitude,AverageDistance (Km),Standard Dev,Density
0,POI1,53.546167,-113.485734,3006.860237,1213.841193,20.397561
1,POI2,53.546167,-113.485734,3006.860237,1213.841193,20.397561
2,POI3,45.521629,-73.566024,1001.674735,265.365606,15.776471
3,POI4,45.224830,-63.232729,889.371033,1634.146194,45.312095


In [770]:
df.head(50)

,TimeSt,Country,Province,City,Latitude,Longitude,Closest POI,Distance(Km)
_ID,,,,,,,,
4516516,2017-06-21 00:00:00.143,CA,ON,Waterloo,43.49347,-80.49123,POI1 or POI2,1381.922185
4516547,2017-06-21 18:00:00.193,CA,ON,London,42.93990,-81.27090,POI1 or POI2,1458.919119
4516550,2017-06-21 15:00:00.287,CA,ON,Guelph,43.57760,-80.22010,POI1 or POI2,1358.479660
4516600,2017-06-21 15:00:00.307,CA,ON,Stratford,43.37160,-80.97730,POI1 or POI2,1423.271675
4516693,2017-06-21 14:00:00.597,CA,ON,Kitchener,43.43810,-80.50990,POI1 or POI2,1384.882063
4516771,2017-06-21 10:00:00.873,CA,ON,Sarnia,42.96100,-82.37300,POI1 or POI2,1544.402638
4516831,2017-06-21 12:00:00.950,CA,ON,London,43.00910,-81.17650,POI1 or POI2,1449.413533
4516953,2017-06-21 16:00:01.700,CA,ON,Kitchener,43.42780,-80.51350,POI1 or POI2,1385.445338
4517044,2017-06-21 21:00:00.327,CA,ON,Windsor,42.28250,-83.03720,POI1 or POI2,1618.569699
